# dEdx weights calculation

## Input

This notebook expects to find a txt file called "VolumesZPosition.txt". This file is coming from the _Validation/Geometry_ package and more details on the setup and the procedure can be found here: 

https://twiki.cern.ch/twiki/bin/view/Sandbox/HGCalMaterialBudget

In [1]:
# Add one line at top for the EE front with Stainless Steel e.g
# StainlessSteel 3190.5 0 0 0 
inputfile = "VolumesZPosition_detailedsimulation.txt"

## Some necessary imports

In [2]:
from collections import OrderedDict
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pylab as plt
import seaborn as sns

## Materials dEdx and radiation lengths

In [3]:
#In MeV/mm
dEdx = OrderedDict()
#--------
#Some elements necessary to build our materials
dEdx['Fe'] = 1.143
dEdx['Mn'] = 1.062
dEdx['Cr'] = 1.046
dEdx['Ni'] = 1.307
dEdx['C']  = 0.3952
dEdx['0']  = 0. # 2.398E-04 -> essentially zero
dEdx['H']  = 0. #3.437E-05 -> essentially zero
dEdx['Br'] = 0. #9.814E-04 -> essentially zero
dEdx['W']  = 2.210
#-------- 

dEdx['Copper'] = 1.257
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#1996
dEdx['H_Scintillator'] = 0.91512109*dEdx['C'] + 0.084878906*dEdx['H']
dEdx['Silicon'] = 0.3876
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2730
dEdx['M_NEMA_FR4_plate'] = 0.18077359*dEdx['Silicon'] + 0.4056325*dEdx['0'] + 0.27804208*dEdx['C'] + 0.068442752*dEdx['H'] + 0.067109079*dEdx['Br']
dEdx['Other'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0290
dEdx['Air'] = 0.
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#3692
dEdx['StainlessSteel'] = 0.6996*dEdx['Fe']+0.01*dEdx['Mn']+0.19*dEdx['Cr']+0.1*dEdx['Ni']+0.0004*dEdx['C'];
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#0568
dEdx['WCu'] = 0.75*dEdx['W']+0.25*dEdx['Copper']
#--------
dEdx['Lead'] = 1.274 #Pb
#Composition of cable as Sunanda uses them is here: 
#http://cmslxr.fnal.gov/source/Geometry/CMSCommonData/data/materials.xml#2841
dEdx['Cables'] = 0.586*dEdx['Copper'] + 0.259*dEdx['C'] + 0.138*dEdx['0'] + 0.017*dEdx['H']

#In mm
MatXo = OrderedDict()
MatXo['Copper'] = 14.3559
MatXo['H_Scintillator'] = 425.393
MatXo['Cables'] = 66.722
MatXo['M_NEMA_FR4_plate'] = 175.056
MatXo['Silicon'] = 93.6762
MatXo['Other'] = 0.
MatXo['Air'] = 301522.
MatXo['StainlessSteel'] = 17.3555
MatXo['WCu'] = 5.1225
MatXo['Lead'] = 5.6118

In [4]:
# Let's read the input file
# Mat is prepoint material, Z is post point material start, so 
# upper edge of prepoint material. E is the energy loss in the 
# prepoint volume. 
matZ = pd.read_csv(inputfile, sep=" ", header=None, names=["Mat", "Z", "Eta", "R", "E"])

In [5]:
# We will make a new column with the physical thickness of the volumes in mm
matZ["PhysThickInmm"] = abs(matZ["Z"].shift(-1) -  matZ["Z"])
matZ["PhysThickInmm"] = matZ["PhysThickInmm"].shift(1)

In [6]:
# We will add a column that indicates the track the relevant volume belongs to. 
# The logic is that right before the next track "PhysThickInmm" column will be 
# very large. 
matZ["trackflag"] = matZ.apply(lambda row: True if row["PhysThickInmm"] < 2000. else False ,axis=1)
matZ["tracknum"] = (( matZ["trackflag"] == False) & (matZ["trackflag"].shift() == True)).cumsum()
matZ["tracknum"] = matZ["tracknum"].shift(1)
matZ = matZ.drop('trackflag', 1)

In [7]:
# Now that we have the tracknum we will not let the last Copper volume to 
# destroy our calculation with its huge thickness due to track change. 
#According to TDR BH back is at 5137.7 mm so we put 6.0 mm for that Copper
matZ.loc[ matZ["PhysThickInmm"] > 2000. , "PhysThickInmm" ] = 6.0

In [8]:
# Again, adding a new column with the physical thickness of the volumes in radiation lengths
matZ["PhysThickInXo"] = matZ.apply(lambda row: row["PhysThickInmm"] / MatXo[row["Mat"]],axis=1)

In [9]:
# Adding a new column with the dEdx of the material that the volumes is build
matZ["dEdx"] = matZ.apply(lambda row: dEdx[row["Mat"]],axis=1)

In [10]:
# Another column with the dEdx times thickness to help us with the calculation of the 
# final dEdx weights 
matZ["dEdxtimesdx"] = matZ["dEdx"] * matZ["PhysThickInmm"]

In [11]:
# And here a column with the cumulative sum
matZ["dEdxtimesdxCum"] = matZ.groupby('tracknum')["dEdxtimesdx"].cumsum()

In [12]:
# And here a column with the cumulative sum for E
matZ["ECum"] = matZ.groupby('tracknum')["E"].cumsum()

In [13]:
# And here a column for the cumulative sum in radiation length 
matZ["PhysThickInXoCum"] = matZ.groupby('tracknum')["PhysThickInXo"].cumsum()

In [14]:
# We will add a column that indicates the layer the relevant volume belongs to. 
# The logic is that if the previous material is Silicon or Scintillator
# we change layer.
matZ["layerflag"] = matZ.apply(lambda row: False if row["Mat"] == "Silicon" or row["Mat"] == "H_Scintillator" else True ,axis=1)
matZ["layer"] = ( matZ["layerflag"] == True) & (matZ["layerflag"].shift(1) == False) 
matZ["layer"] = matZ.groupby('tracknum')["layer"].cumsum()
#The convention is that layers starts from 1
matZ["layer"] = matZ.apply(lambda row: row["layer"] + 1,axis=1)

In [15]:
# Drop auxillary columns
#We need layerflag for not counting the silicon/scintillator in the dedx. 
#matZ = matZ.drop('layerflag', 1)

In [16]:
# This was my misunderstanding. There should be 53 in the layers column since 
# after the last scintillator or silicon the index will increase, there is 
# material there. In case we want to filter them out uncomment the following. 
# Be careful! Filter chooses and not disgards!
# matZ = matZ.groupby('tracknum').filter(lambda g: ~(g['layer'] == 53.0).any()  ) 

In [17]:
#display(matZ)
#matZ[(matZ["layer"] == 1) & matZ["tracknum"] == 20]
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#matZ[(matZ["tracknum"] == 6.0)]
matZ = matZ.dropna()
matZ
#matZ[(matZ["Mat"] == "Air")]
#matZ[ (matZ["Z"] > 3950) & (matZ["Z"] < 4000) & (matZ["Mat"] == "Silicon") & (matZ["R"] < 900)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["tracknum"] == 5) ]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["Z"] < 3900) & (matZ["Z"] > 0) & (matZ["PhysThickInmm"] > 0.22)]
#matZ[ (matZ["Mat"] == "Silicon") & (matZ["layer"] > 28) & (matZ["layer"] < 36)& (matZ["PhysThickInmm"] < 0.22)]
#matZ[matZ["layer"] >= 36]
#matZ[(matZ["Mat"] == "WCu") & (matZ["Z"] > 3950) & (matZ["Z"] < 4000)]

,Mat,Z,Eta,R,E,PhysThickInmm,tracknum,PhysThickInXo,dEdx,dEdxtimesdx,dEdxtimesdxCum,ECum,PhysThickInXoCum,layerflag,layer
1,StainlessSteel,3190.80,2.35502,611.063,0.017209,0.30,0.0,0.017286,1.139861,0.341958,0.341958,0.017209,0.017286,True,1.0
2,Lead,3192.90,2.35502,611.466,0.144835,2.10,0.0,0.374211,1.274000,2.675400,3.017358,0.162044,0.391497,True,1.0
3,Copper,3193.00,2.35502,611.485,0.006278,0.10,0.0,0.006966,1.257000,0.125700,3.143058,0.168322,0.398463,True,1.0
4,StainlessSteel,3193.30,2.35502,611.542,0.017209,0.30,0.0,0.017286,1.139861,0.341958,3.485017,0.185531,0.415748,True,1.0
5,M_NEMA_FR4_plate,3194.90,2.35502,611.849,0.022097,1.60,0.0,0.009140,0.179950,0.287920,3.772937,0.207628,0.424888,True,1.0
6,Air,3196.40,2.35502,612.136,0.000014,1.50,0.0,0.000005,0.000000,0.000000,3.772937,0.207642,0.424893,True,1.0
7,M_NEMA_FR4_plate,3198.00,2.35502,612.442,0.022097,1.60,0.0,0.009140,0.179950,0.287920,4.060857,0.229739,0.434033,True,1.0
8,Silicon,3198.12,2.35502,612.465,0.002140,0.12,0.0,0.001281,0.387600,0.046512,4.107369,0.231879,0.435314,False,1.0
9,Silicon,3198.30,2.35502,612.500,0.003209,0.18,0.0,0.001922,0.387600,0.069768,4.177137,0.235088,0.437236,False,1.0
10,WCu,3199.70,2.35502,612.768,0.133932,1.40,0.0,0.273304,1.971750,2.760450,6.937587,0.369020,0.710540,True,2.0


In [18]:
#matZ[matZ["Z"]>0]
#matZ[matZ["PhysThickInmm"]> 2000.]
#matZ[matZ["tracknum"] == 2 ]
#matZ[ (matZ["tracknum"] == 20) & (matZ["layer"] > 10) ]
#matZ[450:500]

In [24]:
# We will add a column with the dedx weights
# First dedxtimesdx sum per layer but not including silicon or scintillator
matZ = matZ.drop(matZ[ (matZ.Mat == "Silicon") | (matZ.Mat == "H_Scintillator") ].index)
# This is with the theoretical dedx
mdEdxtimesdxsumperlayer = matZ.groupby(['tracknum','layer'])["dEdxtimesdx"].sum()
# And now with the detailed simulation
mdEdxtimesdxsumperlayer_detailed = matZ.groupby(['tracknum','layer'])["E"].sum()
#type(mdEdxtimesdxsumperlayer)
mdEdxtimesdxsumperlayer = mdEdxtimesdxsumperlayer.to_frame().reset_index()
mdEdxtimesdxsumperlayer_detailed = mdEdxtimesdxsumperlayer_detailed.to_frame().reset_index()
#mdEdxtimesdxsumperlayer
#type(mdEdxtimesdxsumperlayer)

#Let's put also the accumulated energy loss
mdEdxtimesdxsumperlayer["dEdxtimesdxCum"] = mdEdxtimesdxsumperlayer.groupby('tracknum')["dEdxtimesdx"].cumsum()
mdEdxtimesdxsumperlayer_detailed["ECum"] = mdEdxtimesdxsumperlayer_detailed.groupby('tracknum')["E"].cumsum()

#Final weights
mdEdxtimesdxsumperlayer["dedxweights"] = (mdEdxtimesdxsumperlayer["dEdxtimesdx"] + mdEdxtimesdxsumperlayer["dEdxtimesdx"].shift(-1))/2
mdEdxtimesdxsumperlayer_detailed["dedxweights_detailedsimulation"] = (mdEdxtimesdxsumperlayer_detailed["E"] + mdEdxtimesdxsumperlayer_detailed["E"].shift(-1))/2

#Hack for the last layer
mdEdxtimesdxsumperlayer.loc[mdEdxtimesdxsumperlayer["layer"] == 52.0, "dedxweights"] = 89.366024
mdEdxtimesdxsumperlayer_detailed.loc[mdEdxtimesdxsumperlayer_detailed["layer"] == 52.0, "dedxweights_detailedsimulation"] = 89.366024

#Drop duplicates not needed columns
mdEdxtimesdxsumperlayer_detailed = mdEdxtimesdxsumperlayer_detailed.drop(['tracknum', 'layer'], axis=1)
#mdEdxtimesdxsumperlayer[ mdEdxtimesdxsumperlayer['tracknum'] == 6.0  ]
#mdEdxtimesdxsumperlayer_detailed[ mdEdxtimesdxsumperlayer_detailed['tracknum'] == 6.0  ]
mdEdxtimesdxsumperlayer = pd.concat([mdEdxtimesdxsumperlayer, mdEdxtimesdxsumperlayer_detailed], axis=1)
mdEdxtimesdxsumperlayer

,tracknum,layer,dEdxtimesdx,dEdxtimesdxCum,dedxweights,E,ECum,dedxweights_detailedsimulation
0,0.0,1.0,4.060857,4.060857,8.561878,0.229739,0.229739,0.437146
1,0.0,2.0,13.062900,17.123757,10.592307,0.644554,0.874293,0.552016
2,0.0,3.0,8.121714,25.245470,10.592307,0.459478,1.333771,0.552016
3,0.0,4.0,13.062900,38.308370,10.592307,0.644554,1.978325,0.552016
4,0.0,5.0,8.121714,46.430084,10.592307,0.459478,2.437803,0.552016
5,0.0,6.0,13.062900,59.492984,10.592307,0.644554,3.082357,0.552016
6,0.0,7.0,8.121714,67.614697,10.592307,0.459478,3.541834,0.552016
7,0.0,8.0,13.062900,80.677597,10.592307,0.644554,4.186388,0.552016
8,0.0,9.0,8.121714,88.799311,10.592307,0.459478,4.645866,0.552016
9,0.0,10.0,13.062900,101.862211,10.592307,0.644554,5.290420,0.552016
